In [0]:
from pyspark.sql.functions import to_date, first, col, round

In [0]:
df_junto = spark.read.parquet("dbfs:/databricks-results/bronze/*/*/*")

In [0]:
moedas = ['USD', 'EUR', 'GBP']

df_moedas = df_junto.filter(df_junto.moeda.isin(moedas))

In [0]:
df_moedas = df_moedas.withColumn("data", to_date("data"))

In [0]:
resultado_taxas_conversao = df_moedas.groupBy("data")\
                                    .pivot("moeda")\
                                    .agg(first("taxa"))\
                                    .orderBy("data", ascending=False)

In [0]:
resultado_valores_reais = resultado_taxas_conversao.select("*")

+----------+--------+--------+--------+
|      data|     EUR|     GBP|     USD|
+----------+--------+--------+--------+
|2023-04-01|0.181681|0.159983|0.197499|
|2023-04-02|0.182756|0.160693| 0.19748|
|2023-04-03|0.180972|0.158945|0.197449|
|2023-04-04|0.179912|0.157752|0.197153|
|2023-04-05|0.182154|0.159374|0.198622|
+----------+--------+--------+--------+
only showing top 5 rows



In [0]:
for moeda in moedas:
    resultado_valores_reais = resultado_valores_reais\
                                .withColumn(
                                    moeda, round(1/col(moeda), 4)  
                                )

In [0]:
resultado_taxas_conversao = resultado_taxas_conversao.coalesce(1)
resultado_valores_reais = resultado_valores_reais.coalesce(1)

In [0]:
# Salvando arquivo de taxas

resultado_taxas_conversao.write\
  .mode("overwrite")\
  .format("csv")\
  .option("header", "true")\
  .save("dbfs:/databricks-results/prata/taxas_conversao")


resultado_valores_reais.write\
  .mode("overwrite")\
  .format("csv")\
  .option("header", "true")\
  .save("dbfs:/databricks-results/prata/valores_reais")

In [0]:
display(dbutils.fs.ls("dbfs:/databricks-results/bronze/2023/06/02"))

path,name,size,modificationTime
dbfs:/databricks-results/bronze/2023/06/02/_SUCCESS,_SUCCESS,0,1685710425000
dbfs:/databricks-results/bronze/2023/06/02/_committed_4783029921312244598,_committed_4783029921312244598,416,1685710425000
dbfs:/databricks-results/bronze/2023/06/02/_started_4783029921312244598,_started_4783029921312244598,0,1685710422000
dbfs:/databricks-results/bronze/2023/06/02/part-00000-tid-4783029921312244598-6a77a0ad-5f55-43c4-95a8-3386dced342c-0-1-c000.snappy.parquet,part-00000-tid-4783029921312244598-6a77a0ad-5f55-43c4-95a8-3386dced342c-0-1-c000.snappy.parquet,1666,1685710425000
dbfs:/databricks-results/bronze/2023/06/02/part-00001-tid-4783029921312244598-6a77a0ad-5f55-43c4-95a8-3386dced342c-1-1-c000.snappy.parquet,part-00001-tid-4783029921312244598-6a77a0ad-5f55-43c4-95a8-3386dced342c-1-1-c000.snappy.parquet,1659,1685710425000
dbfs:/databricks-results/bronze/2023/06/02/part-00002-tid-4783029921312244598-6a77a0ad-5f55-43c4-95a8-3386dced342c-2-1-c000.snappy.parquet,part-00002-tid-4783029921312244598-6a77a0ad-5f55-43c4-95a8-3386dced342c-2-1-c000.snappy.parquet,1677,1685710425000
dbfs:/databricks-results/bronze/2023/06/02/part-00003-tid-4783029921312244598-6a77a0ad-5f55-43c4-95a8-3386dced342c-3-1-c000.snappy.parquet,part-00003-tid-4783029921312244598-6a77a0ad-5f55-43c4-95a8-3386dced342c-3-1-c000.snappy.parquet,1704,1685710425000
